In [9]:
# Libraries
import numpy as np   
import time
import scipy.sparse.linalg as spy
import warnings
warnings.filterwarnings('ignore')


def pos_def(n):
    A = np.random.rand(n,n)
    return (A+A.T) + n*np.eye(n)

n = 20
A = pos_def(n)
x_sol = np.floor(np.random.rand(n) * 100)
b = np.dot(A, x_sol)

alpha = 0.01
M = A + alpha*np.identity(n)

In [125]:
# GMRes without optimization
def gmres(A,b,it=100, tol=1e-5):
    Q = np.zeros((b.shape[0], it+1))
    H = np.zeros((it+1,it))
    x0 = np.zeros((b.shape[0]))
    
    r = b - np.dot(A,x0)
    beta0 = np.linalg.norm(b)
    beta1 = np.linalg.norm(r)
    Q[:,0] = r/beta1
    
    for i in range(it):
        e = np.zeros((i+2))
        e[0] = 1
        
        w = np.dot(A,Q[:,i])
        
        for j in range(i+1):
            h = np.dot(Q[:,j],w)
            w -= h*Q[:,j]
            H[j,i] = h
        
        H[i+1,i] = np.linalg.norm(w)
        
        if H[i+1,i] != 0:
            Q[:,i+1] = w/H[i+1,i]
        
        y,residual,_,_ = np.linalg.lstsq(H[:i+2,:i+1], beta1*e)
        
        if H[i+1,i] == 0 or len(residual) == 0 or (residual/beta0 < tol):# and i+1 >= A.shape[0]):
            break
        
    return np.dot(Q[:,:i+1], y), i

In [126]:
# A better implementation of preconditioned GMRes without Cauchy integral (left)
def left_prec_gmres(A_0,b_0,M_0,it=100,tol=1e-5):
    Q = np.zeros((b_0.shape[0], it+1))
    H = np.zeros((it+1,it))
    x0 = np.zeros((b_0.shape[0]))
    
    # copy
    A = np.copy(A_0)
    b = np.copy(b_0)
    M = np.copy(M_0)
    
    # Left preconditioner -> A and b changes
    b = gmres(M,b)[0]
    
    r = b - np.dot(A,x0)
    beta0 = np.linalg.norm(b)
    beta1 = np.linalg.norm(r)
    Q[:,0] = r/beta1
    
    for i in range(it):
        e = np.zeros((i+2))
        e[0] = 1
        
        w = gmres(M, np.dot(A,Q[:,i]))[0]
        
        for j in range(i+1):
            h = np.dot(Q[:,j],w)
            w -= h*Q[:,j]
            H[j,i] = h
        
        H[i+1,i] = np.linalg.norm(w)
        
        if H[i+1,i] != 0:
            Q[:,i+1] = w/H[i+1,i]
        
        y,residual,_,_ = np.linalg.lstsq(H[:i+2,:i+1], beta1*e)
        
        if H[i+1,i] == 0 or len(residual) == 0 or residual/beta0 < tol:
            break
        
    return np.dot(Q[:,:i+1], y), i

In [127]:
# A better implementation of preconditioned GMRes without Cauchy integral (right)
def right_prec_gmres(A,b,M,it=100,tol=1e-5):
    Q = np.zeros((b.shape[0], it+1))
    H = np.zeros((it+1,it))
    x0 = np.zeros((b.shape[0]))
    
    r = b 
    beta0 = np.linalg.norm(b)
    beta1 = np.linalg.norm(r)
    Q[:,0] = r/beta1
    
    for i in range(it):
        e = np.zeros((i+2))
        e[0] = 1
        
        x = gmres(M, Q[:,i])[0]
        w = np.dot(A, x)
        
        for j in range(i+1):
            h = np.dot(Q[:,j],w)
            w -= h*Q[:,j]
            H[j,i] = h
        
        H[i+1,i] = np.linalg.norm(w)
        
        if H[i+1,i] != 0:
            Q[:,i+1] = w/H[i+1,i]
        
        y,residual,_,_ = np.linalg.lstsq(H[:i+2,:i+1], beta1*e)
        
        if H[i+1,i] == 0 or len(residual) == 0 or residual/beta0 < tol:
            break
    
    x_tild = np.dot(Q[:,:i+1], y)
    return gmres(M, x_tild)[0],i

In [140]:
def trapezoid2(myfun, N, a, b):
    x = np.linspace(0, b, N/2) # We want N bins, so N+1 points  
    h = x[1]-x[0]
    xmiddle = x[1:-1]
    int_val = 0
    for i in xmiddle:
        int_val += (myfun(i) + myfun(-i))
    int_val = myfun(a) + 2*int_val + myfun(b)
    return 0.5*h*int_val

def z(t, c, r):
    return c + r*np.complex(np.cos(t), np.sin(t))

def dz(t, r):
    return r*np.complex(np.cos(t), np.sin(t))

def g(t,l,L,M,v,f, tol=1e-5):
    fz = f(z(t, (l+L)/2., (L-l)/2.))
    dzz = dz(t, (L-l)/2.)
    p = fz*dzz
    #gmr = spy.gmres(z(t, (l+L)/2. - alpha, (L-l)/2.)/p *np.identity(v.shape[0]) - (1./p)*M, v)[0]
    gmr = p*spy.gmres(((l+L)/2. + dzz)*np.identity(v.shape[0]) - A, v)[0]
    return gmr


def cauchy_integral(l, L, M, v, Nf = 50,N=500):
    f = lambda x: (1. - (alpha/x)**(Nf+1))/(x - alpha)
    g1 = lambda t: g(t,l,L,M,v,f)
    val = trapezoid2(g1, N, -np.pi, np.pi) / (2.*np.pi)
    return val

In [141]:
# GMRes using contour integral to compute the preconditioner
def cauchy_prec_gmres(A,b,M,it=100,tol=1e-5):
    Q = np.zeros((b.shape[0], it+1))
    H = np.zeros((it+1,it))
    x0 = np.zeros((b.shape[0]))
    
    r = b 
    beta0 = np.linalg.norm(b)
    beta1 = np.linalg.norm(r)
    Q[:,0] = r/beta1
    
    lamb, vect = np.linalg.eig(M)
    L = np.amax(np.real(lamb))
    l = np.amin(np.real(lamb))
    d = np.amax(np.imag(lamb))
    
    for i in range(it):
        e = np.zeros((i+2))
        e[0] = 1
        
        x = cauchy_integral(l, L, M, Q[:,i])
        w = np.dot(A, x)
        
        for j in range(i+1):
            h = np.dot(Q[:,j],w)
            w -= h*Q[:,j]
            H[j,i] = h
        
        H[i+1,i] = np.linalg.norm(w)
        
        if H[i+1,i] != 0:
            Q[:,i+1] = w/H[i+1,i]
        
        y,residual,_,_ = np.linalg.lstsq(H[:i+2,:i+1], beta1*e)
        if H[i+1,i] == 0 or len(residual) == 0 or residual/beta0 < tol:
            break
    
    x_tild = np.dot(Q[:,:i+1], y)
    x = cauchy_integral(l, L, M, x_tild)
    return x,i

In [142]:
# GMRes using contour integral to compute the preconditioner
def cauchy_left_prec_gmres(A_0,b_0,M_0,it=100,tol=1e-5):
    Q = np.zeros((b_0.shape[0], it+1))
    H = np.zeros((it+1,it))
    x0 = np.zeros((b_0.shape[0]))
    
    A = np.copy(A_0)
    b = np.copy(b_0)
    M = np.copy(M_0)
    
    lamb, vect = np.linalg.eig(M)
    L = np.amax(np.real(lamb))
    l = np.amin(np.real(lamb))
    d = np.amax(np.imag(lamb))
    
    b = cauchy_integral(l, L, M, b)
    
    r = b 
    beta0 = np.linalg.norm(b)
    beta1 = np.linalg.norm(r)
    Q[:,0] = r/beta1
    
    for i in range(it):
        e = np.zeros((i+2))
        e[0] = 1
        
        w = cauchy_integral(l,L,M,np.dot(A,Q[:,i]))
        
        for j in range(i+1):
            h = np.dot(Q[:,j],w)
            w -= h*Q[:,j]
            H[j,i] = h
        
        H[i+1,i] = np.linalg.norm(w)
        
        if H[i+1,i] != 0:
            Q[:,i+1] = w/H[i+1,i]
        
        y,residual,_,_ = np.linalg.lstsq(H[:i+2,:i+1], beta1*e)
        if H[i+1,i] == 0 or len(residual) == 0 or residual/beta0 < tol:
            break
    
    return np.dot(Q[:,:i+1], y), i

In [143]:
print "Exact solution:\n", x_sol
print "\nNp Solve:\n", spy.gmres(A,b)
print "\nNormal GMRes:\n", gmres(A,b)
print "\nLeft Prec NC:\n", left_prec_gmres(A,b,M)
print "\nRight Prec NC:\n", right_prec_gmres(A,b,M)
print "\nCauchy Left Prec:\n", cauchy_left_prec_gmres(A, b, M)
print "\nCauchy Right Prec:\n", cauchy_prec_gmres(A,b,M)

Exact solution:
[ 70.  90.  40.  43.  61.  25.  11.  72.  64.  28.  87.   5.  63.  75.  29.
  76.  39.  64.  59.  11.]

Np Solve:
(array([ 69.99993215,  89.99962821,  40.00014628,  43.00052883,
        61.00067061,  25.00049728,  11.00037716,  72.00085541,
        63.99957936,  27.99955553,  86.99985088,   4.99934368,
        63.00001264,  75.00004051,  28.99911359,  75.99954419,
        39.00041969,  64.00000704,  58.99972911,  11.00016041]), 0)

Normal GMRes:
(array([ 69.99993215,  89.99962821,  40.00014628,  43.00052883,
        61.00067061,  25.00049728,  11.00037716,  72.00085541,
        63.99957936,  27.99955553,  86.99985088,   4.99934368,
        63.00001264,  75.00004051,  28.99911359,  75.99954419,
        39.00041969,  64.00000704,  58.99972911,  11.00016041]), 4)

Left Prec NC:
(array([ 69.99901623,  89.99433787,  40.00615092,  43.00276056,
        61.0039234 ,  25.00946578,  11.00768131,  71.99829953,
        63.99521325,  28.00946713,  86.99319462,   5.01016405,
        